In [8]:
import pickle, json



In [7]:
atis_fn = "/home/yoga/workspace/knodle_hackathon_fork/knodle/hackathon_seq/atis.json"

In [9]:
with open(atis_fn, "rb") as input_file:
    atis_dataset = json.load(input_file, encoding="UTF8")
    

In [10]:
#with open(atis_fn, "rb") as input_file:
#    atis_dataset = pickle.load(input_file, encoding="UTF8")

In [12]:
atis_dataset.keys()

dict_keys(['label_dict', 'train_sents', 'train_labels', 'vocab', 'test_sents', 'test_labels'])

In [23]:
s1 = atis_dataset['train_sents'][0]
word_to_id = atis_dataset['vocab']
id_to_word = {word_to_id[word]:word for word in word_to_id}
id_to_word

{359: 'one',
 310: 'midway',
 17: 'ac',
 369: 'out',
 25: 'airline',
 292: 'm80',
 132: 'dc',
 509: 'train',
 528: 'ua',
 290: 'lunch',
 483: 'then',
 189: 'find',
 269: 'leaves',
 454: 'stands',
 4: "'s",
 385: 'possible',
 52: 'arizona',
 381: 'plane',
 162: 'ea',
 308: 'michigan',
 214: 'goes',
 80: 'breakfast',
 541: 'via',
 323: 'most',
 265: 'latest',
 339: 'nonstop',
 72: 'before',
 564: 'worth',
 228: 'hours',
 342: 'noontime',
 142: 'departures',
 335: 'night',
 349: 'nw',
 139: 'departing',
 266: 'layover',
 371: 'passengers',
 421: 'seating',
 355: 'offers',
 29: 'airport',
 160: 'dulles',
 75: 'book',
 54: 'arrange',
 566: 'y',
 570: 'you',
 77: 'boston',
 234: 'in',
 318: 'month',
 357: 'okay',
 278: 'live',
 333: 'newark',
 499: 'tickets',
 136: 'delta',
 48: 'approximately',
 3: "'re",
 179: 'f',
 38: 'amount',
 171: 'eighth',
 481: 'the',
 410: 'round',
 302: 'meal',
 267: 'least',
 123: 'could',
 8: 'DIGIT',
 219: 'h',
 232: 'i',
 1: "'ll",
 185: 'ff',
 241: 'is',
 58:

In [24]:
[id_to_word[wid] for wid in s1]

['i',
 'want',
 'to',
 'fly',
 'from',
 'boston',
 'at',
 'DIGITDIGITDIGIT',
 'am',
 'and',
 'arrive',
 'in',
 'denver',
 'at',
 'DIGITDIGITDIGITDIGIT',
 'in',
 'the',
 'morning']

In [29]:
l1 = atis_dataset['train_labels'][0]
label_to_id = atis_dataset['label_dict']
id_to_label = {label_to_id[label]:label for label in label_to_id}
id_to_label

{95: 'I-depart_date.today_relative',
 21: 'B-cost_relative',
 40: 'B-flight',
 118: 'I-state_name',
 106: 'I-flight_stop',
 54: 'B-mod',
 47: 'B-fromloc.airport_name',
 58: 'B-restriction_code',
 62: 'B-return_date.month_name',
 107: 'I-flight_time',
 37: 'B-economy',
 30: 'B-depart_date.year',
 68: 'B-state_name',
 25: 'B-depart_date.date_relative',
 2: 'B-airline_name',
 123: 'I-toloc.city_name',
 126: 'O',
 13: 'B-arrive_time.start_time',
 4: 'B-airport_name',
 42: 'B-flight_mod',
 121: 'I-today_relative',
 69: 'B-stoploc.airport_code',
 105: 'I-flight_number',
 56: 'B-or',
 35: 'B-depart_time.time',
 104: 'I-flight_mod',
 9: 'B-arrive_date.today_relative',
 125: 'I-transport_type',
 34: 'B-depart_time.start_time',
 60: 'B-return_date.day_name',
 57: 'B-period_of_day',
 119: 'I-stoploc.city_name',
 53: 'B-meal_description',
 102: 'I-fare_amount',
 31: 'B-depart_time.end_time',
 71: 'B-stoploc.city_name',
 101: 'I-economy',
 81: 'B-toloc.state_name',
 44: 'B-flight_stop',
 76: 'B-tol

In [31]:
[(id_to_word[wid], id_to_label[lid]) for (wid, lid) in zip(s1,l1)]

[('i', 'O'),
 ('want', 'O'),
 ('to', 'O'),
 ('fly', 'O'),
 ('from', 'O'),
 ('boston', 'B-fromloc.city_name'),
 ('at', 'O'),
 ('DIGITDIGITDIGIT', 'B-depart_time.time'),
 ('am', 'I-depart_time.time'),
 ('and', 'O'),
 ('arrive', 'O'),
 ('in', 'O'),
 ('denver', 'B-toloc.city_name'),
 ('at', 'O'),
 ('DIGITDIGITDIGITDIGIT', 'B-arrive_time.time'),
 ('in', 'O'),
 ('the', 'O'),
 ('morning', 'B-arrive_time.period_of_day')]

In [36]:
l1

[126,
 126,
 126,
 126,
 126,
 48,
 126,
 35,
 99,
 126,
 126,
 126,
 78,
 126,
 14,
 126,
 126,
 12]

In [ ]:
# knodle/transformation/majority.py:
#    if not use_probabilistic_labels:
#        # convert labels represented as a prob distribution to a single label using majority voting
#        kwargs = {"choose_random_label": True, "other_class_id": other_class_id}
#        noisy_y_train = np.apply_along_axis(probabilies_to_majority_vote, axis=1, arr=noisy_y_train, **kwargs)
